In this notebook, two datasets namely two_sides and chemical_substr are cleaned and merged into a single dataframe prior to fitting the models

## Import Libraries

In [1]:
# Import libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
#pd.set_option('display.max_columns', None)  
import seaborn as sns
#pd.set_option('display.max_rows', None)
#from tqdm._tqdm_notebook import tqdm_notebook as tqdm
#from tqdm import tqdm

## Import two_sides dataset - drug-drug interaction dataset

In [2]:
#Import two sides dataset. TWOSIDES - data of side effects of pairs of drugs

two_sides_full=pd.read_csv('data/TWOSIDES.csv.xz.csv', compression='xz', \
                      header=0, sep=',', quotechar='"')


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2,4,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
two_sides_full.head(2)

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
0,10355,Temazepam,136411,sildenafil,10003239,Arthralgia,7,149,24,1536,2.91667,0.421275,0.0448718
1,1808,Bumetanide,7824,Oxytocin,10003239,Arthralgia,1,13,2,138,5.0,1.19224,0.0714286


In [13]:
two_sides_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42920391 entries, 0 to 42920390
Data columns (total 13 columns):
drug_1_rxnorn_id            object
drug_1_concept_name         object
drug_2_rxnorm_id            object
drug_2_concept_name         object
condition_meddra_id         object
condition_concept_name      object
A                           object
B                           object
C                           object
D                           object
PRR                         object
PRR_error                   object
mean_reporting_frequency    object
dtypes: object(13)
memory usage: 4.2+ GB


In [4]:
two_sides_full.query('(drug_1_concept_name=="Tamoxifen") & \
(drug_2_concept_name=="sildenafil") &(condition_concept_name=="Diarrhoea")')

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency


In [5]:
two_sides_full.query('(drug_1_concept_name=="Acetaminophen") & \
(drug_2_concept_name=="glycopyrronium") &(condition_concept_name=="Nausea")')

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
79,161,Acetaminophen,1546438,glycopyrronium,10028813,Nausea,3,20,6,224,5.0,0.672439,0.130435
1549,161,Acetaminophen,1546438,glycopyrronium,10028813,Nausea,3,20,6,224,5.0,0.672439,0.130435


In [6]:
#Drop the unwanted columns 
two_sides=two_sides_full.drop(['A', 'B', 'C', 'D', 'PRR', 'PRR_error', \
                          'mean_reporting_frequency', 'drug_1_rxnorn_id', \
                         'drug_2_rxnorm_id', 'condition_meddra_id'], axis=1)

In [7]:
#rename columns
two_sides.columns=['drug_1', 'drug_2', 'side_effects']

In [8]:
two_sides.head(1)

,drug_1,drug_2,side_effects
0,Temazepam,sildenafil,Arthralgia


In [9]:
#There are duplicates for certain drug pairs: remove duplicates
two_sides=two_sides.drop_duplicates(subset=['drug_1', 'drug_2', 'side_effects'])

In [10]:
# to find unique drug-drug pair
#drug_withoutcondn_duplicates=two_sides.drop_duplicates(subset=['drug_1', 'drug_2'])

In [11]:
#drug_withoutcondn_duplicates.info()

In [12]:
two_sides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42919391 entries, 0 to 42920390
Data columns (total 3 columns):
drug_1          object
drug_2          object
side_effects    object
dtypes: object(3)
memory usage: 1.3+ GB


In [14]:
two_sides.side_effects.nunique()

12726

two_sides.query('(drug_1=="Temazepam") & \
(drug_2=="sildenafil") &(condn_name=="Diarrhoea")')

Since there are 42 million observations with many side effects and for sake of computing power, only the most common side effects were selected for model building. 

This creates a selection bias and will lead to imbalance dataset as shown in other notebooks. 

In [15]:
two_sides_r=two_sides.query('side_effects=="Nausea" | side_effects =="Dyspnoea" | \
side_effects =="Diarrhoea" | side_effects =="Vomiting" | side_effects =="Pyrexia" | \
side_effects =="Pneumonia"')

In [16]:
#After selecting for the 6 most common side effects, there were 926,781 observations
two_sides_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 926781 entries, 3 to 42920304
Data columns (total 3 columns):
drug_1          926781 non-null object
drug_2          926781 non-null object
side_effects    926781 non-null object
dtypes: object(3)
memory usage: 28.3+ MB


In [17]:
two_sides_r.head(1)

,drug_1,drug_2,side_effects
3,Tamoxifen,Prednisone,Diarrhoea


In [18]:
two_sides_r.drug_2.nunique()

1805

In [20]:
two_sides_r.drug_1.nunique()

1677

In [34]:
#dummize the columns
#pd.get_dummies(df, columns=['key2']).groupby(['key1'], as_index=False).sum()
#pd.concat([df, pd.get_dummies(df.ANNOT)], 1).groupby(['CHR','BP','SNP','CM']).sum()
two_sides_r1=pd.concat([two_sides_r, pd.get_dummies(two_sides_r.side_effects)], 1)\
#.groupby(['drug_1', 'drug_2']).reset_index()
#.sum()
#two_sides_r1=pd.get_dummies(two_sides_r, columns=['condn_name'])

In [23]:
#convert the drug names to lower case. Converted to lower case to match with the drug names
#in chemical substructure dataset (see below under "import chemical structure dataset")
two_sides_r1['drug_1']=two_sides_r1.drug_1.str.lower()
two_sides_r1['drug_2']=two_sides_r1.drug_2.str.lower()

In [21]:
two_sides_r1.iloc[:,2:]=two_sides_r1.iloc[:,2:].apply(pd.to_numeric, downcast='integer')

In [35]:
two_sides_r1.head(2)

,drug_1,drug_2,side_effects,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting
3,Tamoxifen,Prednisone,Diarrhoea,1,0,0,0,0,0
4,Temazepam,sildenafil,Diarrhoea,1,0,0,0,0,0


In [36]:
two_sides_r1.nunique()

drug_1          1677
drug_2          1805
side_effects       6
Diarrhoea          2
Dyspnoea           2
Nausea             2
Pneumonia          2
Pyrexia            2
Vomiting           2
dtype: int64

In [24]:
two_sides_r1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204571 entries, 0 to 204570
Data columns (total 8 columns):
drug_1       204571 non-null object
drug_2       204571 non-null object
Diarrhoea    204571 non-null int8
Dyspnoea     204571 non-null int8
Nausea       204571 non-null int8
Pneumonia    204571 non-null int8
Pyrexia      204571 non-null int8
Vomiting     204571 non-null int8
dtypes: int8(6), object(2)
memory usage: 4.3+ MB


In [25]:
two_sides_r1.to_csv('data/two_sides_r1', index=False)

In [28]:
#create a list of drugs from drug_1 column
two_sides_drug1list=two_sides_r1.drug_1.tolist()
type(two_sides_drug1list)

list

In [ ]:
#create a list of drugs from drug_2 column
two_sides_drug2list=two_sides_r1.drug_2.tolist()

# Import Chemical Structure Dataset

In [29]:
#Import chemical substr dataset from pawel's dataset
chem_str=pd.read_csv('data/chemical_substr.txt', sep='\t')

In [30]:
#rename column 1
chem_str=chem_str.rename(columns={"Unnamed: 0": "drug_name"})

In [31]:
chem_str.head(2)

,drug_name,SUB1,SUB2,SUB3,SUB4,SUB5,SUB6,SUB7,SUB8,SUB9,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
0,carnitine,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,GABA,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
chem_str['drug_name']=chem_str.drug_name.str.lower()

In [33]:
chem_str.iloc[:,1:889]=chem_str.iloc[:,1:889].apply(pd.to_numeric, downcast='integer')

In [34]:
chem_str.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Columns: 882 entries, drug_name to SUB881
dtypes: int8(881), object(1)
memory usage: 771.1+ KB


# Merges
##Merge1- merge drug_1 from two_sides to chem_str drug_name and get the chem substr
##merge 2 - merge drug_2 from two_sides to chem_str drug_name to get their chem substr
Step 3- create list of each of the chem substr from merge 1 and merge 2 separately
step 4 - find maximum of the binary profiles
step 5 - Merge the file from step 4 back to combination of merge 1 and 2

### MERGE1
Steps: 

Step 1: merge drug-pair with side effects with chem str

step 2: drop all the columns except the substr

step 3- create a dataframe that has only the drug-drug pair for the corresponding substructure profile. this will be used to merge with merge2 dataset to get the correct drug-drug pair and the corresponding substructures

In [35]:
#Merge1
tscs_merge1=pd.merge(two_sides_r1, chem_str, left_on='drug_1', right_on='drug_name', \
                     how='inner')

In [36]:
#tscs_merge1=tscs_merge1.drop(['drug_2'], axis=1)
#.drop(['drug_name']), axis=1

In [37]:
tscs_merge1.head(2)

,drug_1,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting,drug_name,SUB1,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
0,acarbose,acebutolol,0,1,0,0,1,0,acarbose,1,...,0,0,0,0,0,0,0,0,0,0
1,acarbose,acenocoumarol,1,1,0,0,1,1,acarbose,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tscs_merge1_drugname= tscs_merge1.iloc[:, 0:2]

In [ ]:
tscs_merge1_drugname.head(2)

In [ ]:
tscs_merge1.iloc[123069:123073, ]

In [ ]:
tscs_merge1.info()

In [ ]:
tscs_merge1.drug_1.nunique()

In [ ]:
tscs_merge1_list1=tscs_merge1.iloc[:, 9:]
tscs_merge1_list=tscs_merge1_list1.values.tolist()

#tscs_merge1_list=tcsc.reset_index()[['index', 'Names', 'Births']].values.astype(str).tolist()

tscs_merge1_list=pd.Series(tscs_merge1_list)

In [ ]:
tscs_merge1_list1.head(2)

In [ ]:
tscs_merge1_list[0:3]

### MERGE2

In [ ]:
#Merge2
tscs_merge2=pd.merge(two_sides_r1, chem_str, left_on='drug_2', right_on='drug_name', \
                     how='inner')

In [ ]:
#tscs_merge2=tscs_merge2.drop(['drug_1'], axis=1)

In [ ]:
tscs_merge2.head(2)

In [ ]:
tscs_merge2_substr= tscs_merge2.drop(['drug_1', 'Diarrhoea', 'Dyspnoea', 'Nausea', \
                                      'Pneumonia', 'Pyrexia', 'Vomiting', 'drug_name'], \
                                     axis=1)

In [ ]:
tscs_merge2_substr.head(2)

### merge the drug_2 with substr to tscs_merge1_drugname to make both lists similar to original drug-drug pair before doing np.maximum function

In [ ]:
tscs_merge2_substr_merge1_drugname=pd.merge(tscs_merge2_substr, tscs_merge1_drugname, \
                                           on='drug_2', how='inner')

In [ ]:
tscs_merge2_substr_merge1_drugname.head(2)

In [ ]:
tscs_merge2_str_list2=tscs_merge2_substr.iloc[:, 1:]
tscs_merge2_str_list2a=tscs_merge2_str_list2.values.tolist()
tscs_merge2_str_list2a=pd.Series(tscs_merge2_str_list2a)

In [ ]:
tscs_merge2_str_list2a.head(2)

In [ ]:
#tscs_merge2_list

In [ ]:
#Merge 3 - Merge drug_2 from tscs_merge2 with drug_2 of tscs_merge1 
#tscs_merge3=pd.merge(tscs_merge1_substr, tscs_merge2_substr, on='drug_2', \
#                     how='inner')

In [ ]:
#tscs_m1_drugstr=tscs_merge3.iloc[:, 2:884]

In [ ]:
#tscs_m1_drugstr.head(1)

In [ ]:
#tscs_m1_drugstr_list=tscs_m1_drugstr.values.tolist()
#tscs_m1_drugstr_list=pd.Series(tscs_m1_drugstr_list)

In [ ]:
#tscs_m1_drugstr=tscs_merge3.iloc[:, 9:890]
#tscs_merge2_list=tscs_merge2_list2.values.tolist()
#tscs_merge2_list=pd.Series(tscs_merge2_list)

In [ ]:
tscs_merge2.iloc[123069:, ]

In [ ]:
len(tscs_merge2_list[123068])

In [ ]:
type(tscs_merge2_list)

### step 4 - find maximum of the binary profiles

In [ ]:
chem_str_combind=np.maximum(tscs_merge1_list[0:123069], tscs_merge2_list[0:123069])

In [ ]:
len(chem_str_combind[0])

In [ ]:
type(chem_str_combind)

In [ ]:
#convert the ADE_str list to dataframe 
chem_str_combind=pd.DataFrame(chem_str_combind.tolist())

In [ ]:
chem_str_combind.head(2)

In [ ]:
#add suffix to column names
chem_str_combind=chem_str_combind.add_suffix('_sub')

#convert to int8
chem_str_combind=chem_str_combind.apply(pd.to_numeric, downcast='integer')
#type(se_binary_ade)

In [ ]:
chem_str_combind.head(2)

### Merge drug fields with max chem_str binary profile(chem_str_combind)

In [ ]:
tscs_merge1_drug_se=tscs_merge1.iloc[:123069, :9]
tscs_merge1_drug_se.info()

In [ ]:
tscs_merge1_drug_se.head(2)

In [ ]:
final_model_data=pd.concat([tscs_merge1_drug_se, chem_str_combind], axis=1)

In [ ]:
final_model_data.head()

In [ ]:
final_model_data=final_model_data.drop(['drug_name'],axis=1)

In [ ]:
#write the final model data to csv
final_model_data.to_csv('data/final_model_data', index=False)

In [ ]:
#convert series to dataframe

In [ ]:
#chem_drug1_twosides=chem_str[chem_str['drug_name'].isin(two_sides_drug1list)]

#chem_drug2_twosides=chem_str[chem_str['drug_name'].isin(two_sides_drug2list)]

In [ ]:
chem_drug1_twosides.info()